In [10]:
import pandas as pd
import urllib.request

url = "https://raw.githubusercontent.com/justmarkham/pycon-2016-tutorial/master/data/sms.tsv"
urllib.request.urlretrieve(url, "sms.tsv")

df = pd.read_csv("sms.tsv", sep='\t', header=None, names=['label', 'message'])
print(df.head())


  label                                            message
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...


In [11]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Map labels to binary values
df['label_num'] = df.label.map({'ham': 0, 'spam': 1})

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(df['message'], df['label_num'], test_size=0.2, random_state=42)

# Vectorize text data
vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Train Logistic Regression model
model = LogisticRegression(max_iter=1000)
model.fit(X_train_tfidf, y_train)

# Predict & evaluate
y_pred = model.predict(X_test_tfidf)
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(classification_report(y_test, y_pred, target_names=['Ham', 'Spam']))

# Test custom samples
samples = [
    "Congratulations! You've won a free ticket to Bahamas. Call now!",
    "Hey, are we still meeting for dinner tonight?",
    "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text to enter."
]
samples_tfidf = vectorizer.transform(samples)
predictions = model.predict(samples_tfidf)
for text, pred in zip(samples, predictions):
    print(f"\nText: {text}\nPredicted Label: {'Spam' if pred else 'Ham'}")


Accuracy: 0.9695
              precision    recall  f1-score   support

         Ham       0.97      1.00      0.98       966
        Spam       1.00      0.77      0.87       149

    accuracy                           0.97      1115
   macro avg       0.98      0.89      0.93      1115
weighted avg       0.97      0.97      0.97      1115


Text: Congratulations! You've won a free ticket to Bahamas. Call now!
Predicted Label: Ham

Text: Hey, are we still meeting for dinner tonight?
Predicted Label: Ham

Text: Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text to enter.
Predicted Label: Spam
